In [16]:
from stocksymbol import StockSymbol
from scraper import get_financial_information
from ssk import ssk
import pandas as pd
import yfinance as yf

In [24]:
ssm = StockSymbol(ssk)

In [14]:
# Get tickers of companies listed in NASDAQ
symbol_list = ssm.get_symbol_list(market="US")
nasdaq_list = [x['symbol'] for x in symbol_list if x['exchange'] == 'NASDAQ']

In [37]:
def get_ratios(ticker):
    financial_info = get_financial_information(ticker)

    financial_info['ebtda-margin'] = financial_info['ebitda'].astype(float) / financial_info['revenue'].astype(float)
    financial_info['gross-profit-margin'] = financial_info['gross-profit'].astype(float) / financial_info['revenue'].astype(float)
    financial_info['cogs-margin'] = financial_info['cost-goods-sold'].astype(float) / financial_info['revenue'].astype(float)

    return financial_info[['ebtda-margin', 'gross-profit-margin', 'cogs-margin', 'eps-basic-net-earnings-per-share', 'eps-earnings-per-share-diluted' ]]

In [57]:
def normalize(df):

    max_vals = df.max()
    min_vals = df.min()

    result = {}
    for col in df.columns:
        df[col] = df[col].astype(float)
        print(col)
        print(df[col])
        result[col] = df[col].apply(lambda x: (x - min_vals[col])/max_vals[col])
        print(result)

    return result

In [58]:
normalize(get_ratios('AAPL'))

TypeError: cannot convert the series to <class 'float'>